<a href="https://colab.research.google.com/github/roy-sr/AI_public/blob/master/NASNetLarge_Categorisation_Action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet_v2 import ResNet152V2, preprocess_input, decode_predictions
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions
from tensorflow.keras.applications.nasnet import NASNetLarge, preprocess_input, decode_predictions
from tensorflow.keras.layers import Input, Dense, AveragePooling2D, GlobalAveragePooling2D, Input, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
import os
import shutil
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix , precision_score, recall_score, accuracy_score, f1_score, accuracy_score
from glob import glob
import pandas as pd
from numpy import asarray
from numpy import savetxt
from datetime import datetime




**Variales**

In [0]:
ImageFile.LOAD_TRUNCATED_IMAGES = True  # For PIL lib issue
epochs = 60
batch_size = 32
input_shape = (331, 331,3)
catg_name = "action"
myFolderName = catg_name.capitalize()

**Datasets**

In [4]:
model_dataset = "/content/drive/My Drive/Categorization/saved_model"
if not os.path.exists(model_dataset):
    os.makedirs(model_dataset)
    print(model_dataset , " has been created.")
else:
  print(model_dataset , " is present.")


home = "/content/drive/My Drive/Categorization"
if not os.path.exists(home):
    os.makedirs(home)
    print(home , " has been created.")
else:
  print(home , " is present.")


tb_dataset = "/content/drive/My Drive/Categorization/tb/" + catg_name
if not os.path.exists(tb_dataset):
    os.makedirs(tb_dataset)
    print(tb_dataset , " has been created.")
else:
  print(tb_dataset , " is present.")


cm_folder = home + "/cm/"+ catg_name
if not os.path.exists(cm_folder):
    os.makedirs(cm_folder)
    print(cm_folder , " has been created.")
else:
  print(cm_folder , " is present.")


model_save_folder = model_dataset +"/"+ catg_name
if not os.path.exists(model_save_folder):
    os.makedirs(model_save_folder)
    print(model_save_folder , " has been created.")
else:
  print(model_save_folder , " is present.")
  

/content/drive/My Drive/Categorization/saved_model  is present.
/content/drive/My Drive/Categorization  is present.
/content/drive/My Drive/Categorization/tb/action  is present.
/content/drive/My Drive/Categorization/cm/action  is present.
/content/drive/My Drive/Categorization/saved_model/action  is present.


**Extract using Unzip
**

In [0]:
#!unzip '/content/drive/My Drive/classification/{myFolderName}.zip' -d '/content/drive/My Drive/Categorization/{catg_name}/'
#!unzip "/content/drive/My Drive/classification/Action.zip" -d "/content/drive/My Drive/Categorization/input/action"

In [5]:
os.chdir("/content/drive/My Drive/Categorization/input/"+ catg_name)
dir_list = [name for name in os.listdir(".") if os.path.isdir(name)]

input_dataset = "/content/drive/My Drive/Categorization/input/"+ catg_name + "/" +dir_list[0]  
print("Input Dataset is ",input_dataset)
print("Please check and match the above input Dataset folder. It should match with the folder where the files has been extracted in the previous step.")

Input Dataset is  /content/drive/My Drive/Categorization/input/action/Action
Please check and match the above input Dataset folder. It should match with the folder where the files has been extracted in the previous step.


**Model Selection**

In [0]:
def get_model(input, output_classes):

  nasnetlarge = NASNetLarge(include_top=False, weights='imagenet') 
  for l in nasnetlarge.layers:
    l.trainable = False

  nasnetlarge_t = NASNetLarge(include_top=True, weights="imagenet")
  for l in nasnetlarge_t.layers:
    l.trainable = True
  avg_pool_layer = nasnetlarge_t.layers[-2]   

  x = nasnetlarge(input)
  x = avg_pool_layer(x)
  out = Dense(output_classes, activation='softmax')(x)
  model = Model(inputs=input, outputs=out)
  del nasnetlarge_t
  return model


**DataGenerator for NasNetLarge**

In [7]:

datagen  = ImageDataGenerator(validation_split=0.2,
                              rotation_range=40,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              rescale=1./255,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest')

#datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2)  


train_generator = datagen.flow_from_directory(input_dataset,
                                                 target_size = (331, 331),
                                                 batch_size = batch_size,
                                                 class_mode = "categorical",
                                                 shuffle=True,
                                                 subset="training")

test_generator = datagen.flow_from_directory(input_dataset,
                                            target_size = (331, 331),
                                            batch_size = batch_size,
                                            class_mode = "categorical",
                                            shuffle=True,
                                            subset="validation")



Found 13575 images belonging to 41 classes.
Found 3374 images belonging to 41 classes.


**Number of Output classes**

In [8]:
out_classes = len(train_generator.class_indices.keys())
print("Total output classes : ", out_classes)

Total output classes :  41


**Model Initialization** - *NasNetLarge*

In [9]:
input = Input(shape=input_shape)

nasnetlarge_model = get_model(input,out_classes)
nasnetlarge_model.summary()
nasnetlarge_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

359751680/359748576 [==============================] - 7s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 331, 331, 3)]     0         
_________________________________________________________________
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
global_average_pooling2d (Gl (None, 4032)              0         
_________________________________________________________________
dense (Dense)                (None, 41)                165353    
Total params: 85,082,171
Trainable params: 165,353
Non-trainable params: 84,916,818
_________________________________________________________________


**Load from Checkpoint**

In [10]:
model_save_file = model_save_folder + "/"+catg_name+"_nasnetlarge_best.h5"
if os.path.exists(model_save_file):
  nasnetlarge_model.load_weights(model_save_file)
  print("Loaded")
else:
  print("No checkpoint found")

Loaded


**Execute Training**

In [0]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
model_save_file = model_save_folder + "/"+catg_name+"_nasnetlarge_best.h5"

tb_dataset = tb_dataset + "/nasnetlarge_catg"

if not os.path.exists(tb_dataset):
    os.makedirs(tb_dataset)

tensor_board = TensorBoard(log_dir=tb_dataset, histogram_freq=0, write_graph=True, write_images=True)

checkpoint = ModelCheckpoint(model_save_file, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_categorical_accuracy', mode='max', patience=3, restore_best_weights = True)

nasnetlarge_model.fit(
        train_generator,
        batch_size = batch_size,
        epochs=epochs,
        validation_data=test_generator,
        validation_batch_size =  batch_size,
        callbacks=[tensor_board, checkpoint,early_stopping] )

Epoch 1/60
363/425 [========================>.....] - ETA: 13:57 - loss: 1.3135 - categorical_accuracy: 0.6219

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


425/425 [==============================] - ETA: 0s - loss: 1.3063 - categorical_accuracy: 0.6223 
Epoch 00001: val_categorical_accuracy improved from -inf to 0.59544, saving model to /content/drive/My Drive/Categorization/saved_model/action/action_nasnetlarge_best.h5
425/425 [==============================] - 7184s 17s/step - loss: 1.3063 - categorical_accuracy: 0.6223 - val_loss: 1.3821 - val_categorical_accuracy: 0.5954
Epoch 2/60
425/425 [==============================] - ETA: 0s - loss: 1.1045 - categorical_accuracy: 0.6721
Epoch 00002: val_categorical_accuracy improved from 0.59544 to 0.59899, saving model to /content/drive/My Drive/Categorization/saved_model/action/action_nasnetlarge_best.h5
425/425 [==============================] - 1523s 4s/step - loss: 1.1045 - categorical_accuracy: 0.6721 - val_loss: 1.3848 - val_categorical_accuracy: 0.5990
Epoch 3/60
425/425 [==============================] - ETA: 0s - loss: 1.0219 - categorical_accuracy: 0.6894
Epoch 00003: val_categorical

In [0]:
dt_time = datetime.now().strftime('%Y%m%d%H%M%S')
#target = model_save_folder + "/"+catg_name+"_nasnetlarge_"+dt_time+".h5"
target = os.path.join(model_save_folder, catg_name+"_nasnetlarge_"+dt_time+".h5")
shutil.copyfile(model_save_file, target)

'/content/drive/My Drive/Categorization/saved_model/action/action_nasnetlarge_20200609150542.h5'

In [0]:
#nasnetlarge_model = keras.models.load_model(model_dataset+"/"+catg_name+"/"+catg_name+"_nasnetlarge.h5")

In [0]:
Y_pred = nasnetlarge_model.predict(test_generator, test_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

In [15]:
#dt_time = "20200609150542"  # Please check this form the output of model save cells if you are running at different time than that of the save 
#Confusion Matrix 
cm_nasnetlarge = confusion_matrix(test_generator.classes, y_pred)
#os.chdir(input_dataset) """" class_names = [name for name in os.listdir(".") if os.path.isdir(name)]

class_names = list(test_generator.class_indices.keys()) 
cm_df = pd.DataFrame(data=cm_nasnetlarge,columns=class_names)
cm_df['0'] = class_names
class_names = np.insert(class_names, 0, '0', axis=0)
cm_df = cm_df[class_names]  
cm_df.to_csv(path_or_buf = cm_folder + "/cm_nasnetlarge_catg_"+dt_time+".csv",header=True)
#========Accuracy=========================#
stat_text= ""
accuracy = accuracy_score(test_generator.classes, y_pred)
stat_text = stat_text + " Accuracy : " + str(accuracy)
#=========== PRFS ========================#
prfs = precision_recall_fscore_support(test_generator.classes, y_pred, average='micro')
prfs = "precision_recall_fscore_support_nasnetlarge : " + str(prfs)
stat_text = stat_text + " | " + prfs
print(stat_text)
with open(cm_folder + "/statText_nasnetlarge_catg_"+dt_time+".txt", "w") as text_file:
    text_file.write(stat_text)


 Accuracy : 0.04208654416123296 | precision_recall_fscore_support_nasnetlarge : (0.04208654416123296, 0.04208654416123296, 0.04208654416123296, None)


In [31]:
unique_elements, counts_elements = np.unique(test_generator.classes, return_counts=True) 
print("Frequency of unique values of the said array:")
print(np.asarray(( counts_elements)))


Frequency of unique values of the said array:
[ 35   7  79  58 144  44 330   5  38  63 153  58  59  62  13 139 129  71
  56 112 159  97  77  76  73  48 118  78  62  66 168 148  53  58  36  43
  68  37  40 164  50]


In [32]:
unique_elements, counts_elements = np.unique(y_pred, return_counts=True) 
print("Frequency of unique values of the said array:")
print(np.asarray(( counts_elements)))


Frequency of unique values of the said array:
[ 15   1  83  42 118  38 420   4  30  84 138  37  22  44   8 106 165  85
  77 100 156 130 114  63  63  38 161 101  58  32 169 141  70  27  33  60
  59  19  32 187  44]
